In [1]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import (
    FullOptimStateDictConfig,
    FullStateDictConfig,
)
# import neptune.new as neptune
import neptune
import transformers
from datetime import datetime
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(
        offload_to_cpu=True, rank0_only=False
    ),
)
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

/home/alexander/venvs/enp2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# run = neptune.init_run(
#     api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzYjBkZDNhYS01ODIwLTRlNzQtOGE3Mi00Mzk0YmY5YjgzODgifQ==",
#     project="alshlyapin/train-distilgpt2",
# )
# run["sys/tags"].add(["transformers", "language-modeling"])

In [3]:
os.environ["NEPTUNE_API_TOKEN"] = (
    "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzYjBkZDNhYS01ODIwLTRlNzQtOGE3Mi00Mzk0YmY5YjgzODgifQ=="
)
os.environ["NEPTUNE_PROJECT"] = "alshlyapin/train-distilgpt2"

In [4]:
train_dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")
eval_dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="validation")
test_dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="test")

In [5]:
print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['text'],
    num_rows: 1801350
})
Dataset({
    features: ['text'],
    num_rows: 3760
})
Dataset({
    features: ['text'],
    num_rows: 4358
})


In [6]:
train_dataset[1]

{'text': ' = Valkyria Chronicles III = \n'}

In [7]:
train_dataset[100]

{'text': ' 96 ammunition packing boxes \n'}

In [8]:
base_model_id = "distilbert/distilgpt2"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto")

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id, model_max_length=512, padding_side="left", add_eos_token=True
)
tokenizer.pad_token = tokenizer.eos_token

In [10]:
tokenizer("test")

{'input_ids': [9288], 'attention_mask': [1]}

In [11]:
model_inputs = tokenizer("Transformers are", return_tensors="pt").to("cuda")

output = model.generate(
    **model_inputs,
    max_new_tokens=100,
    pad_token_id=2,
    eos_token_id=tokenizer.eos_token_id
)
output

tensor([[41762,   364,   389,   257,  1049,   835,   284,   651,   534,  2832,
           319,   257,   649,   983,    13,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198]], device='cuda:0')

In [12]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

Transformers are a great way to get your hands on a new game.


























































































In [13]:
def tokenize_func(examples):
    return tokenizer(examples["text"], truncation=True)


encoded_train_dataset = train_dataset.map(tokenize_func, batched=True)
encoded_eval_dataset = eval_dataset.map(tokenize_func, batched=True)

In [14]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


## Run training

In [15]:
project = "train-wikitext"
base_model_name = "distilbert/distilgpt2"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name


tokenizer.pad_token = tokenizer.eos_token


trainer = transformers.Trainer(
    model=model,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_eval_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=10,
        gradient_accumulation_steps=4,
        max_steps=100,
        learning_rate=2.5e-5,
        logging_steps=50,
        fp16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",
        save_strategy="steps",
        save_steps=50,
        evaluation_strategy="steps",
        eval_steps=50,
        do_eval=True,
        report_to="neptune",
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    tokenizer=tokenizer,
)
model.config.use_cache = False
trainer.train()

[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/alshlyapin/train-distilgpt2/e/TRAIN-4


 50%|█████     | 50/100 [02:22<02:25,  2.91s/it]

{'loss': 5.4106, 'learning_rate': 1.3684210526315791e-05, 'epoch': 0.0}


                                                
 50%|█████     | 50/100 [03:37<02:25,  2.91s/it] 

...

{'eval_loss': 4.386573791503906, 'eval_runtime': 75.7357, 'eval_samples_per_second': 49.646, 'eval_steps_per_second': 6.206, 'epoch': 0.0}


100%|██████████| 100/100 [07:19<00:00,  2.67s/it]

{'train_runtime': 443.4426, 'train_samples_per_second': 9.02, 'train_steps_per_second': 0.226, 'train_loss': 5.028552551269531, 'epoch': 0.0}
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 11 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 11 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/alshlyapin/train-distilgpt2/e/TRAIN-4/metadata


100%|██████████| 100/100 [07:20<00:00,  4.40s/it]


TrainOutput(global_step=100, training_loss=5.028552551269531, metrics={'train_runtime': 443.4426, 'train_samples_per_second': 9.02, 'train_steps_per_second': 0.226, 'train_loss': 5.028552551269531, 'epoch': 0.0})

In [16]:
with torch.no_grad():
    print(
        tokenizer.decode(
            model.generate(**model_inputs, max_new_tokens=100, pad_token_id=2)[0],
            skip_special_tokens=True,
        )
    )

/home/alexander/venvs/enp2/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Transformers are a great way to get a better understanding of the game and how to play it.





















































































In [17]:
trainer.save_model("20240222_distilgpt2_100")